In [1]:
import pandas as pd
import numpy as np
import ast

df = pd.read_csv('Recipe_raw_csv/full_dataset.csv', index_col= 0, nrows = 15000)

In [2]:
def data_prep(csv_path, nrows = 20000):
    '''Load and Process the source'''
    
    df = pd.read_csv(csv_path, index_col= 0, nrows = nrows) # Load data without "Unnamed:0"
    
    # preprocess our main column used in queries later
    unwanted = {"'": '', '"': '', ',': '', '\n': ' ', '\t': ' ', '\r': ' '}
    cleaned_list = []


    for x in df['NER']:
        ingredients = ast.literal_eval(x.replace('\n', ''))
        
        # Clean each ingredient
        processed = [
            ing.translate(str.maketrans(unwanted))
               .lower()
               .replace(' ', '_')
               .strip('_')
            for ing in ingredients
        ]
        
        # Join into single string
        cleaned_list.append(' '.join(processed))
            
    df['NER'] = cleaned_list
    
    return df

In [3]:
# df = data_prep('Recipe_raw_csv/full_dataset.csv', nrows= 20000)

In [4]:
import pandas as pd
import sqlite3

# Load the first 20,000 rows (replace with your actual CSV or DataFrame slice)
df = data_prep('Recipe_raw_csv/full_dataset.csv', nrows= 20000)

# Connect to SQLite
conn = sqlite3.connect("recipes.db")
cur = conn.cursor()

# Create base table
cur.execute('''
CREATE TABLE IF NOT EXISTS recipes (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT,
    ingredients TEXT,
    directions TEXT,
    link TEXT,
    source TEXT,
    ner TEXT
)
''')

# Create FTS5 virtual table for ner
cur.execute('''
CREATE VIRTUAL TABLE IF NOT EXISTS recipes_fts 
USING fts5(ner, content='recipes', content_rowid='id')
''')

conn.commit()

# Insert data in batch
batch = df.copy()
batch.to_sql('recipes', conn, if_exists='append', index=False)

# Insert into FTS table
# Get last inserted rowid
cursor = conn.execute('SELECT last_insert_rowid()')
last_id = cursor.fetchone()[0]
first_id = last_id - len(batch) + 1

fts_data = [(i + first_id, row['NER']) for i, row in batch.iterrows()]
cur.executemany("INSERT INTO recipes_fts(rowid, ner) VALUES (?, ?)", fts_data)
conn.commit()


In [9]:
conn.execute('''
             Select * from recipes limit 10
             ''').fetchall()

[(1,
  'No-Bake Nut Cookies',
  '["1 c. firmly packed brown sugar", "1/2 c. evaporated milk", "1/2 tsp. vanilla", "1/2 c. broken nuts (pecans)", "2 Tbsp. butter or margarine", "3 1/2 c. bite size shredded rice biscuits"]',
  '["In a heavy 2-quart saucepan, mix brown sugar, nuts, evaporated milk and butter or margarine.", "Stir over medium heat until mixture bubbles all over top.", "Boil and stir 5 minutes more. Take off heat.", "Stir in vanilla and cereal; mix well.", "Using 2 teaspoons, drop and shape into 30 clusters on wax paper.", "Let stand until firm, about 30 minutes."]',
  'www.cookbooks.com/Recipe-Details.aspx?id=44874',
  'Gathered',
  'brown_sugar milk vanilla nuts butter bite_size_shredded_rice_biscuits'),
 (2,
  "Jewell Ball'S Chicken",
  '["1 small jar chipped beef, cut up", "4 boned chicken breasts", "1 can cream of mushroom soup", "1 carton sour cream"]',
  '["Place chipped beef on bottom of baking dish.", "Place chicken on top of beef.", "Mix soup and cream together; p

In [10]:
query = """
SELECT r.title, r.ingredients 
FROM recipes r 
JOIN recipes_fts f ON r.id = f.rowid 
WHERE f.ner MATCH 'chicken garlic'
LIMIT 5;
"""

results = pd.read_sql_query(query, conn)
print(results)


                                              title  \
0                                    Summer Chicken   
1                                 French Onion Soup   
2                             Smoked Turkey Risotto   
3  Zucchini In Tomato Juice(From Weight Watchers)     
4                                     Egg Drop Soup   

                                         ingredients  
0  ["1 pkg. chicken cutlets", "1/2 c. oil", "1/3 ...  
1  ["1/4 lb. butter", "2 1/2 lb. onions, sliced",...  
2  ["3 c. chicken stock or vegetable stock", "1 p...  
3  ["zucchini", "12 oz. tomato juice", "dash of o...  
4  ["2 Tbsp. peanut oil", "1 c. chopped onion", "...  


In [11]:
query = """
EXPLAIN QUERY PLAN
SELECT r.title 
FROM recipes r 
JOIN recipes_fts f ON r.id = f.rowid 
WHERE f.ner MATCH 'chicken garlic';
"""

plan = pd.read_sql_query(query, conn)
print(plan)


   id  parent  notused                                        detail
0   3       0        0               SCAN f VIRTUAL TABLE INDEX 0:M0
1   8       0        0  SEARCH r USING INTEGER PRIMARY KEY (rowid=?)


In [12]:
cursor = conn.execute("""
EXPLAIN QUERY PLAN
SELECT r.title 
FROM recipes r 
JOIN recipes_fts f ON r.id = f.rowid 
WHERE f.ner MATCH 'chicken garlic'
""")
for row in cursor.fetchall():
    print(row)


(3, 0, 0, 'SCAN f VIRTUAL TABLE INDEX 0:M0')
(8, 0, 0, 'SEARCH r USING INTEGER PRIMARY KEY (rowid=?)')


In [13]:
cursor = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print(tables)


[('recipes',), ('sqlite_sequence',), ('recipes_fts',), ('recipes_fts_data',), ('recipes_fts_idx',), ('recipes_fts_docsize',), ('recipes_fts_config',)]
